In [1]:
import sympy
import numpy as np
import isotropic_tensors as it

from IPython.display import display

# Summary
This notebook shows how to use the code in [isotropic_tensors.py](isotropic_tensors.py) to do linear algebra with isotropic tensors as explained in Section 4 of (insert arxiv link)

High dimensional isotropic tensors can all be represented in index notation through combinations of Kronecker-Delta $\delta_{ij}$ symbols and Levi-Cevita symbols $\epsilon_{ijk}$. Different contractions between isotropic tensors can in principle be evaluated analytically by contracting these index representations. However, this becomes very cumbersome for the very high dimensional objects that we need here.

Therefore, the code in isotropic_tensors.py creates numerical numpy representations of tensors which we can then easily contract computationally. We can then find a representation of the result of some product, by decomposing it again in some isotropic basis. For convenience we additionally use the computer algebra system "sympy" to represent some results in a convenient symbolic form

This notebook shows examples:
1. How the numeric representations of isotropic tensors look
2. How products between tensors are evaluated
3. How we can compute the algebra of isotropic tensors
4. How we can use to define the covariance matrices and their pseudo-inverse

# Isotropic Tensors

In [2]:
its = it.IsotropicTensorSystem(usecache=False)

In [3]:
# We can access some symbolic tensor through a string
# Each tensor has a numerical representation and a sympy symbol representing it
# and some additional properties
print(its("J2=2").N())

[[[[ 0.66666667  0.          0.        ]
   [ 0.         -0.33333333  0.        ]
   [ 0.          0.         -0.33333333]]

  [[ 0.          0.5         0.        ]
   [ 0.5         0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.5       ]
   [ 0.          0.          0.        ]
   [ 0.5         0.          0.        ]]]


 [[[ 0.          0.5         0.        ]
   [ 0.5         0.          0.        ]
   [ 0.          0.          0.        ]]

  [[-0.33333333  0.          0.        ]
   [ 0.          0.66666667  0.        ]
   [ 0.          0.         -0.33333333]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.5       ]
   [ 0.          0.5         0.        ]]]


 [[[ 0.          0.          0.5       ]
   [ 0.          0.          0.        ]
   [ 0.5         0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.5       ]
   [ 0.          0.5         0.        

In [4]:
# Groups of symmetric indices, Norm of the tensor
print(its("J2=2").symshape, its("J2=2").norm2())
# Sympy symbol
display(its("J22").symbol())

(2, 2) 5


J_22

Here some examples of the index representations of different isotropic tensors. $S$ indicates a symmetrization between the indicated indices.

| Tensor          | Index Representation                                                                            | Norm              |
|-----------------|------------------------------------------------------------------------------------------------------------------|-------------------|
| $J_{2}$         | $S_{2}(\delta_{ij})$                                                                            | $3$               |
| $J_{22}$        | $S_{22}(\delta_{ij}\delta_{kl})$                                                                | $9$               |
| $J_{2=2}$       | $- \frac{S_{22}(\delta_{ij}\delta_{kl})}{3} + S_{22}(\delta_{ik}\delta_{jl})$                   | $5$               |
| $J_{4}$         | $S_{4}(\delta_{ij}\delta_{kl})$                                                                 | $5$               |
| $J_{3-3}$       | $S_{33}(\delta_{ij}\delta_{kl}\delta_{mn})$                                                     | $\frac{25}{3}$    |
| $J_{3\equiv3}$  | $- \frac{3 S_{33}(\delta_{ij}\delta_{kl}\delta_{mn})}{5} + S_{33}(\delta_{ik}\delta_{mj}\delta_{ln})$ | $7$               |
| $J_{24}$        | $S_{24}(\delta_{ij}\delta_{kl}\delta_{mn})$                                                     | $15$              |
| $J_{2=4}$       | $- \frac{S_{24}(\delta_{ij}\delta_{kl}\delta_{mn})}{3} + S_{24}(\delta_{ik}\delta_{jl}\delta_{mn})$ | $\frac{35}{6}$    |

## Tensor Algebra
Products between different tensors are simply evaluated through their numerical representations. For example

In [5]:
Jres = np.einsum("ijkl,kl", its("J22").N(), its("J2").N())
print(Jres)

[[3. 0. 0.]
 [0. 3. 0.]
 [0. 0. 3.]]


Corresponding to 3 times the unit tensor. We can use the function its.decompose to decompose the numeric form of some tensor in another basis

In [6]:
# Decompose Jres in basis of orthogonal isotropic tensors with 2 indices (only the unit tensor)
print(its.decompose(Jres, "2"))
# A more complicated example, decompose I2=2 in terms of J22 and J2=2
print(its.decompose(its("I2=2").N(), "22"))
# We also have a function to show symbolic representations
its.symbol_decompose(its("I2=2").N(), "22")

[3.]
[0.33333333 1.        ]


J_22/3 + J_2=2

Here some examples of a few more complicated products:

In [7]:
# np.tensordot is like an einsum over "axes" axes
Ja = np.tensordot(its("J2=2").N(), its("J2=2").N(), axes=2)
display(its.symbol_decompose(Ja, "22"))
Jb = np.tensordot(its("J2=2").N(), its("J22").N(), axes=2)
display(its.symbol_decompose(Jb, "22"))
Jc = np.tensordot(its("J22").N(), its("J24").N(), axes=2)
display(its.symbol_decompose(Jc, "24"))
Jd = np.tensordot(its("J4=4").N(), its("J4=4").N(), axes=4)
display(its.symbol_decompose(Jd, "44"))

J_2=2

0

3*J_24

7*J_4=4/6

Here a systematic overview of 2-index, 3-index and 4-index products that lead to non-zero results:

In [8]:
display(sympy.Matrix(its.symbol_algebra(product=2)).T)
display(sympy.Matrix(its.symbol_algebra(product=3)).T)
display(sympy.Matrix(its.symbol_algebra(product=4)).T)
# All substitutions that make sense based on this algebra are summarized in its.orth_algebra

Matrix([
[J_2**2, J_2*J_22, J_2*J_24, J_2*J_222, J_2*J_2=22, J_22**2, J_22*J_24, J_22*J_222, J_22*J_2=22, J_2=2**2, J_2=2*J_2=4, J_2=2*J_2=22, J_2=2*J_2-2-2-, J_42*J_24, J_4=2*J_2=4],
[     3,    3*J_2,    3*J_4,    3*J_22,      J_2=2,  3*J_22,    3*J_24,    3*J_222,      J_2=22,    J_2=2,       J_2=4,   2*J_2=22/3,       J_2-2-2-,    3*J_44,       J_4=4]])

Matrix([
[ J_3-3**2, J_3\equiv3**2],
[5*J_3-3/3,    J_3\equiv3]])

Matrix([
[J_4**2, J_4*J_42, J_4*J_44, J_24*J_44, J_2=4*J_4=4, J_44**2,  J_4=4**2, J_4==4**2],
[     5,    5*J_2,    5*J_4,    5*J_24,   7*J_2=4/6,  5*J_44, 7*J_4=4/6,    J_4==4]])

Here the relations between the simple basis ($I$-tensors) versus the orthogonalized basis ($J$-tensors):

In [9]:
display(sympy.Matrix(its.symbol_JtoI()).T)
display(sympy.Matrix(its.symbol_ItoJ()).T)

Matrix([
[1, J_2, J_4, J_6, J_8, J_22,           J_2=2, J_3-3,              J_3\equiv3, J_42,           J_4=2, J_24,           J_2=4, J_44,           J_4=4,                         J_4==4, J_222,            J_2=22,                      J_2-2-2-],
[1, I_2, I_4, I_6, I_8, I_22, -I_22/3 + I_2=2, I_3-3, -3*I_3-3/5 + I_3\equiv3, I_42, -I_42/3 + I_4=2, I_24, -I_24/3 + I_2=4, I_44, -I_44/3 + I_4=4, 3*I_44/35 - 6*I_4=4/7 + I_4==4, I_222, -I_222/3 + I_2=22, I_2-2-2- + 2*I_222/9 - I_2=22]])

Matrix([
[1, I_2, I_4, I_6, I_8, I_22,          I_2=2, I_3-3,             I_3\equiv3, I_42,          I_4=2, I_24,          I_2=4, I_44,          I_4=4,                      I_4==4, I_222,           I_2=22,                    I_2-2-2-],
[1, J_2, J_4, J_6, J_8, J_22, J_22/3 + J_2=2, J_3-3, 3*J_3-3/5 + J_3\equiv3, J_42, J_42/3 + J_4=2, J_24, J_24/3 + J_2=4, J_44, J_44/3 + J_4=4, J_44/5 + 6*J_4=4/7 + J_4==4, J_222, J_222/3 + J_2=22, J_2-2-2- + J_222/9 + J_2=22]])

# Covariance Matrices
To write the high dimensional Gaussian distribution of the tidal tensor, we need to write its covariance matrix in terms of Isotropic tensors. Conveniently the Covariance matrix is a full symmetric (in all indices) isotropic tensor. For example:

In [10]:
J4_in22 = its.symbol_decompose(its("J4"), "22") 
print("Decomposition of J4 in the (2,2) symmetric basis:")
display(J4_in22)
print("The covariance matrix of the tidal tensor is exactly J4 * sigma**2/5 -- as explained in the paper")
display(its.covariance_matrix((2,), decomposed=True)[0])

Decomposition of J4 in the (2,2) symmetric basis:


5*J_22/9 + 2*J_2=2/3

The covariance matrix of the tidal tensor is exactly J4 * sigma**2/5 -- as explained in the paper


Matrix([[sigma0**2*J_22/9 + 2*sigma0**2*J_2=2/15]])

In [11]:
print("Here the covariance matrix of third derivatives of the potential")
display(its.covariance_matrix((3,), decomposed=True)[0])
print("Second and fourth derivatives:")
display(its.covariance_matrix((2,4), decomposed=True)[0])
print("Second, third and fourth:")
display(its.covariance_matrix((2,3,4), decomposed=True)[0]) # Note that the covariance between even and uneven derivatives is zero

Here the covariance matrix of third derivatives of the potential


Matrix([[3*sigma1**2*J_3-3/25 + 2*sigma1**2*J_3\equiv3/35]])

Second and fourth derivatives:


Matrix([
[  sigma0**2*J_22/9 + 2*sigma0**2*J_2=2/15,                           -sigma1**2*J_24/15 - 4*sigma1**2*J_2=4/35],
[-sigma1**2*J_42/15 - 4*sigma1**2*J_4=2/35, sigma2**2*J_44/25 + 24*sigma2**2*J_4=4/245 + 8*sigma2**2*J_4==4/315]])

Second, third and fourth:


Matrix([
[  sigma0**2*J_22/9 + 2*sigma0**2*J_2=2/15,                                                0,                           -sigma1**2*J_24/15 - 4*sigma1**2*J_2=4/35],
[                                        0, 3*sigma1**2*J_3-3/25 + 2*sigma1**2*J_3\equiv3/35,                                                                   0],
[-sigma1**2*J_42/15 - 4*sigma1**2*J_4=2/35,                                                0, sigma2**2*J_44/25 + 24*sigma2**2*J_4=4/245 + 8*sigma2**2*J_4==4/315]])

# Pseudo inverse Covariance Matrices
To write the [degenrate multivariate distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Degenerate_case) of the tidal tensor, we need to infer the [generalized inverse](https://en.wikipedia.org/wiki/Generalized_inverse) $\mathbf{C}^+$ of the covariance matrix $\mathbf{C}$. It has the property

$\mathbf{C} \mathbf{C}^+ \mathbf{C} = \mathbf{C}$

A generalized inverse can be found by making a parametric Ansatz for $\mathbf{C}^+$ and then solving the corresponding system of equations. Here, we only show that the generalized inverse that we calculated in the code has indeed this property

In [12]:
CTT, sigma = its.pseudo_inverse_covariance_matrix((2,))
display(CTT)

Matrix([[J_22/sigma0**2 + 15*J_2=2/(2*sigma0**2)]])

In [13]:
sig = 3. # Can put anything you like here
C_TT = (sig**2/5.) * its("J4").N()
C_TTinv = its("J22").N() * (1./sig**2) + its("J2=2").N() * (15./2./sig**2)

Cnew = np.einsum("abij,ijkl,klcd", C_TT, C_TTinv, C_TT)

print("The difference is:")
print(np.max(np.abs(C_TT - Cnew)), "  (~0 up to floating point errors)") 

The difference is:
6.661338147750939e-16   (~0 up to floating point errors)


Here the other generalized inverses, feel free to check them yourself:

In [14]:
display(its.pseudo_inverse_covariance_matrix((3,))[0])
sig1 = 1. # Can put anything you like here
C_SS = its("J6").N() * sig1**2 / 7.
C_SSinv = its("J3-3").N() * (3./sig1**2) + its("J3---3").N() * (35./2./sig1**2)
Cnew = np.einsum("abcijk,ijklmn,lmndef", C_SS, C_SSinv, C_SS)

print(np.max(np.abs(C_SS - Cnew)), "  (~0 up to floating point errors)")

Matrix([[3*J_3-3/sigma1**2 + 35*J_3\equiv3/(2*sigma1**2)]])

1.1102230246251565e-16   (~0 up to floating point errors)


In [15]:
# For the 4-2 covariance block-matrix it is not so easy to get it into numpy format
# we'll work with a symbolic representation of the tensor algebra
CTR, sigs = its.covariance_matrix((2,4))
CTRinv, sigs = its.pseudo_inverse_covariance_matrix((2,4))
display(CTR)
display(CTRinv)

Matrix([
[  sigma0**2*J_22/9 + 2*sigma0**2*J_2=2/15,                           -sigma1**2*J_24/15 - 4*sigma1**2*J_2=4/35],
[-sigma1**2*J_42/15 - 4*sigma1**2*J_4=2/35, sigma2**2*J_44/25 + 24*sigma2**2*J_4=4/245 + 8*sigma2**2*J_4==4/315]])

Matrix([
[15*sigma2**2*J_2=2/(2*sigma0**2*sigma2**2 - 2*sigma1**4) + sigma2**2*J_22/(sigma0**2*sigma2**2 - sigma1**4),                            15*sigma1**2*J_2=4/(2*sigma0**2*sigma2**2 - 2*sigma1**4) + sigma1**2*J_24/(sigma0**2*sigma2**2 - sigma1**4)],
[15*sigma1**2*J_4=2/(2*sigma0**2*sigma2**2 - 2*sigma1**4) + sigma1**2*J_42/(sigma0**2*sigma2**2 - sigma1**4), 15*sigma0**2*J_4=4/(2*sigma0**2*sigma2**2 - 2*sigma1**4) + sigma0**2*J_44/(sigma0**2*sigma2**2 - sigma1**4) + 315*J_4==4/(8*sigma2**2)]])

In [16]:
# its.orth_algebra contains all the symbolic substitutions that we can make
# based on a precomputed algebra as explained above 
CTR_CTRinv = sympy.expand(CTR *  CTRinv).subs(its.orth_algebra)
display(CTR_CTRinv)

Matrix([
[                         3*sigma0**2*sigma2**2*J_22/(9*sigma0**2*sigma2**2 - 9*sigma1**4) + 2*sigma0**2*sigma2**2*J_2=2/(2*sigma0**2*sigma2**2 - 2*sigma1**4) - 5*sigma1**4*J_22/(15*sigma0**2*sigma2**2 - 15*sigma1**4) - 14*sigma1**4*J_2=2/(14*sigma0**2*sigma2**2 - 14*sigma1**4), -5*sigma0**2*sigma1**2*J_24/(15*sigma0**2*sigma2**2 - 15*sigma1**4) - 14*sigma0**2*sigma1**2*J_2=4/(14*sigma0**2*sigma2**2 - 14*sigma1**4) + 3*sigma0**2*sigma1**2*J_24/(9*sigma0**2*sigma2**2 - 9*sigma1**4) + 2*sigma0**2*sigma1**2*J_2=4/(2*sigma0**2*sigma2**2 - 2*sigma1**4)],
[84*sigma1**2*sigma2**2*J_4=2/(98*sigma0**2*sigma2**2 - 98*sigma1**4) + 5*sigma1**2*sigma2**2*J_42/(25*sigma0**2*sigma2**2 - 25*sigma1**4) - 3*sigma1**2*sigma2**2*J_42/(15*sigma0**2*sigma2**2 - 15*sigma1**4) - 12*sigma1**2*sigma2**2*J_4=2/(14*sigma0**2*sigma2**2 - 14*sigma1**4),        84*sigma0**2*sigma2**2*J_4=4/(98*sigma0**2*sigma2**2 - 98*sigma1**4) + 5*sigma0**2*sigma2**2*J_44/(25*sigma0**2*sigma2**2 - 25*sigma1**4) - 3*sigma1

In [17]:
Cnew = sympy.expand(CTR_CTRinv * CTR).subs(its.orth_algebra)
display(Cnew)

Matrix([
[                                                                                                                                                              4*sigma0**4*sigma2**2*J_2=2/(30*sigma0**2*sigma2**2 - 30*sigma1**4) + 3*sigma0**4*sigma2**2*J_22/(27*sigma0**2*sigma2**2 - 27*sigma1**4) - 28*sigma0**2*sigma1**4*J_2=2/(210*sigma0**2*sigma2**2 - 210*sigma1**4) - 15*sigma0**2*sigma1**4*J_22/(135*sigma0**2*sigma2**2 - 135*sigma1**4), 15*sigma0**2*sigma1**2*sigma2**2*J_24/(225*sigma0**2*sigma2**2 - 225*sigma1**4) - 5*sigma0**2*sigma1**2*sigma2**2*J_24/(75*sigma0**2*sigma2**2 - 75*sigma1**4) - 8*sigma0**2*sigma1**2*sigma2**2*J_2=4/(70*sigma0**2*sigma2**2 - 70*sigma1**4) - 3*sigma0**2*sigma1**2*sigma2**2*J_24/(45*sigma0**2*sigma2**2 - 45*sigma1**4) + 8*sigma1**6*J_2=4/(70*sigma0**2*sigma2**2 - 70*sigma1**4) + 3*sigma1**6*J_24/(45*sigma0**2*sigma2**2 - 45*sigma1**4)],
[15*sigma0**2*sigma1**2*sigma2**2*J_42/(225*sigma0**2*sigma2**2 - 225*sigma1**4) - 5*sigma0**2*sigma1**2*sigma

In [18]:
# Show that this equals CTR
sympy.simplify(Cnew - CTR)

Matrix([
[0, 0],
[0, 0]])

In [19]:
# To get a latex representation, we can do for example:
print(sympy.printing.latex(its.pseudo_inverse_covariance_matrix((2,4))[0]))

\left[\begin{matrix}\frac{15 \sigma_{2}^{2} J_{2=2}}{2 \sigma_{0}^{2} \sigma_{2}^{2} - 2 \sigma_{1}^{4}} + \frac{\sigma_{2}^{2} J_{22}}{\sigma_{0}^{2} \sigma_{2}^{2} - \sigma_{1}^{4}} & \frac{15 \sigma_{1}^{2} J_{2=4}}{2 \sigma_{0}^{2} \sigma_{2}^{2} - 2 \sigma_{1}^{4}} + \frac{\sigma_{1}^{2} J_{24}}{\sigma_{0}^{2} \sigma_{2}^{2} - \sigma_{1}^{4}}\\\frac{15 \sigma_{1}^{2} J_{4=2}}{2 \sigma_{0}^{2} \sigma_{2}^{2} - 2 \sigma_{1}^{4}} + \frac{\sigma_{1}^{2} J_{42}}{\sigma_{0}^{2} \sigma_{2}^{2} - \sigma_{1}^{4}} & \frac{15 \sigma_{0}^{2} J_{4=4}}{2 \sigma_{0}^{2} \sigma_{2}^{2} - 2 \sigma_{1}^{4}} + \frac{\sigma_{0}^{2} J_{44}}{\sigma_{0}^{2} \sigma_{2}^{2} - \sigma_{1}^{4}} + \frac{315 J_{4==4}}{8 \sigma_{2}^{2}}\end{matrix}\right]
